In [5]:
import logging
import os
import pandas as pd

from datetime import datetime
from logging.handlers import TimedRotatingFileHandler

import numpy as np

wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)


from dart.api import Dart_main_api
from dart.utils import get_jinja_yaml_conf, create_db_engine, Postgres_connect
from dart.processing import Disc_processor, Report_processor

In [6]:
conf = get_jinja_yaml_conf('./conf/api.yml', './conf/logging.yml')
end_date = datetime.now().date() 


# logger 설정
logger = logging.getLogger('main')
logging.basicConfig(level=eval(conf['logging']['level']),
    format=conf['logging']['format'],
    handlers = [TimedRotatingFileHandler(filename =  conf['logging']['file_name'],
                                when=conf['logging']['when'],
                                interval=conf['logging']['interval'],
                                backupCount=conf['logging']['backupCount']), logging.StreamHandler()]
                )


In [7]:
# Only for notebooks
import re

os.environ['_ts'] = datetime.astimezone(datetime.now()).strftime('%Y-%m-%d %H:%M:%S %z')

with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

In [8]:
engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)
dart_api = Dart_main_api(auth_key = os.environ['auth_key'])

2024-02-18 00:29:24,472 (utils.py 54) INFO ::: Connect to 172.20.10.3. DB_NAME is stocks
2024-02-18 00:29:24,472 (api.py 23) INFO ::: ### dart main api is initialized! ###


# Check

In [10]:
data = postgres_conn.get_data(schema_name= 'dart',
                      table_name = 'br_prvcptl_use')

In [18]:
data2 = data.drop_duplicates()

In [22]:
data[data.duplicated(keep = False)].sort_values(by = ['보고서코드']).iloc[

,접수번호,법인구분,고유번호,회사명,구분,회차,납입일,납입금액,실제자금사용현황,실제자금사용내역_내용,실제자금사용내역_금액,차이발생사유등,자금사용계획,자금사용계획_사용용도,자금사용계획_조달금액,_ts,사업연도,보고서코드
503,20170330002013,E,00100656,강원일보,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2016,11011
504,20170330002013,E,00100656,강원일보,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2016,11011
505,20180330002038,E,00100656,강원일보,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2017,11011
506,20180330002038,E,00100656,강원일보,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2017,11011
191773,20210331002429,K,01351080,지아이이노베이션,제3자배정 유상증자\n(전환우선주),-,2019-07-24,NaN,-,(주6),200000000.0,(주6),-,(주4),200000000.0,2024-02-17 04:17:09.207279+00:00,2020,11011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,20151113000483,Y,00102140,경인전자,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2015,11014
1301,20151113000483,Y,00102140,경인전자,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2015,11014
1316,20221111000307,Y,00102140,경인전자,-,-,None,NaN,-,-,NaN,-,-,-,NaN,2024-02-17 04:17:09.207279+00:00,2022,11014
191721,20231114001456,K,01351080,지아이이노베이션,제3자배정 유상증자\n(전환우선주),-,2019-07-24,NaN,-,주4),500000000.0,-,-,주4),500000000.0,2024-02-17 04:17:09.207279+00:00,2023,11014
